In [ ]:
import numpy as np
import pandas as pd
import xlrd
import openpyxl

import os
file_name = '成品采购明细'
folder_name = '成品采购明细'

import sys
month=' ('+str(sys.argv[1])+'月)'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)

input_filename = file_name +'.xls'
output_filename = folder_name + '/'+ file_name+ '_添加类别.xlsx'

df = pd.read_excel(input_filename, sheet_name = 0)
df.columns.values

In [ ]:
# 去除 出库统计 金额
tag_names = list(df.iloc[0])
print(tag_names,'\n')
col_names = list(df.columns.values)
print(col_names,'\n')

#idx_typename = tag_names.index('类别名称')
#str_typename = col_names[idx_typename]
#print(str_typename,'\n')

idx_delivery = col_names.index('出库统计')
str_delivery_1= col_names[idx_delivery]
str_delivery_2 = col_names[idx_delivery +1]

print(str_delivery_1)
print(str_delivery_2)

df.drop([str_delivery_1,str_delivery_2], axis=1, inplace = True)


In [ ]:
#names = df.loc[0,:]
#print(names[names == '类别名称'].index)
#print(names)

In [ ]:
#重命名"类别名称"所在列 的标签
tag_names = list(df.iloc[0])
col_names = list(df.columns.values)

idx_typename = tag_names.index('类别名称')
str_typename = col_names[idx_typename]
print(str_typename,'\n')

df.rename(index=str, columns={str_typename: 'type_name'},inplace=True)
df.type_name.unique()

str_typesize = col_names[idx_typename + 1]
print(str_typesize, '\n')

df.rename(index=str, columns={str_typesize: 'pd_size'}, inplace = True)

# 添加一列，命名为 type
print(df.columns.values,'\n')
df.insert(idx_typename,'type',df.type_name)
df.columns.values

In [ ]:
df.type[df.type_name == '类别名称'] = '类别'

df.type[(df.type_name =='HDF') | (df.type_name == 'HDG') | (df.type_name == 'HDJ') | (df.type == 'HDT') ] = '地砖'
df.type[df.type_name =='HTF'] = '地砖'
df.type[df.type_name == 'XF'] = '地砖'

df.type[(df.type_name =='HNF') | (df.type_name == 'HNG') | (df.type_name == 'HNJ') | (df.type == 'HNT') ] = '瓷片'
df.type[df.type_name == 'PG'] = '瓷片'
df.type[(df.type_name == 'W') | (df.type_name == 'WM')] = '瓷片'
df.type[(df.type_name == 'XG') | (df.type_name == 'XT') | (df.type_name == 'XW')] = '瓷片'

df.type[df.type_name =='HM'] = '木纹砖'
df.type[df.type_name == 'HW'] = '微晶'
df.type[df.type_name == 'HY'] = '全抛釉'
df.type[df.type_name == 'H3S'] = '色晶砖'
df.type[df.type_name == 'HS'] = '大理石'
df.type[df.type_name == 'HNW'] = '镜面砖'
df.type[(df.type_name == 'P') | (df.type_name == 'T')] = '通体砖'
df.type[df.type_name == '浮雕'] = '浮雕'
df.type[df.type_name == '文具'] = '五金仓'
df.type[df.type_name == '木托'] = '木托'
df.type[df.type_name == '捆砖带'] = '捆砖带'
df.type[df.type_name == '换包装'] = '换包装'
df.type[df.type_name == '花片'] = '花片'
df.type[df.type_name == '欧驰'] = '欧驰'
df.type[df.type_name == '腰线'] = '腰线'
df.type[df.type_name == '纸箱'] = '纸箱'
df.type[df.type_name == '瓷砖胶'] = '5KG'
df.type[df.type_name == '圣陶居'] = '圣陶居'
df.type[df.type_name == '展美'] = '展美'

In [ ]:
print(df.groupby('type').type.agg('count'))

In [ ]:
#modified output
output_df = df.copy(deep=True)
output_df.columns = output_df.iloc[0,:]
output_df.drop(['0'], axis=0,inplace = True)
output_df.set_index('单号',inplace = True)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
output_df.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory

In [ ]:
df2 = output_df.copy(deep=True)
df2.replace(np.nan,'',regex = True, inplace = True)
output_file_list = list(df2['供应商名'].unique())
output_file_list

In [ ]:
def handle_worksheet_format(writer, group, title):
    # handle the spreadsheet format
    workbook = writer.book
    worksheet = writer.sheets['sheet1']

    # set data format
    for idx, col in enumerate(group):  # loop through all columns
        series = group[col]
        if isinstance(series[0],str): char_ratio = 2 
        else: char_ratio = 1 
        max_len = max((
                        series.astype(str).map(len).max() * char_ratio ,  # len of largest item
                        len( str(series.name) * 2)  # len of column name/header
                    )) + 2  # adding a little extra space

        data_fmt = workbook.add_format({'num_format': '0.00',
                                        'align': 'right'})    
        #print(series.name, char_ratio, max_len, series.astype(str).map(len).max() * char_ratio, 
        #      len( str(series.name) * 2), series[0] )
        worksheet.set_column(idx+1, idx+1, max_len, data_fmt)  # set column width

    # set index format
    series = group.index
    max_len_idx = max((
            series.astype(str).map(len).max() * 2,  # len of largest item
            len(str(series.name)* 2)  # len of column name/header
            ))+2
    index_fmt = workbook.add_format()
    index_fmt.set_align('right')
    worksheet.set_column(0,0, max_len_idx, index_fmt)

    # set title 
    header_format = workbook.add_format({'bold': True,
                                     'text_wrap': True,
                                     'align': 'center',
                                     'valign': 'vcenter',
                                     'fg_color': '#D7E4BC', 
                                     'border': 1}) 
    
    worksheet_cols = group.shape[1]
    range_str = 'A1:'+chr(ord('A')+worksheet_cols)+'1'
    worksheet.merge_range(range_str,title ,header_format)

In [ ]:

for key, sub_df in df2.groupby('供应商名'):
    if key != '':
        print(key)
        sub_df = sub_df.apply(pd.to_numeric,errors='ignore')
        group = pd.concat([ sub_df.groupby('类别')['金额'].agg(np.sum),
                              sub_df.groupby('类别')['加工金额'].agg(np.sum),
                              sub_df.groupby('类别')['装车金额'].agg(np.sum),
                              sub_df.groupby('类别')['纸箱金额'].agg(np.sum),
                              sub_df.groupby('类别')['总金额'].agg(np.sum)],
                            axis = 1)
        
        summation = group.sum()
        summation.name = '总计'
        group = group.append(summation)
        
        # round up to 2 digits precision
        group= group.round(2)
        
        output_filename = folder_name+'/summary_'+key+'.xlsx'
        writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
        group.to_excel(writer,startrow = 1, sheet_name= 'sheet1') #creates workbook 
               
        # redefine spreadsheet format
        title = key + month
        handle_worksheet_format(writer, group, title)
        
        writer.save() #saves workbook to file in python file directory
        writer.close()
    

In [ ]:
# 入库统计
grouped = df2.groupby('供应商名')['总金额'].agg(np.sum).to_frame()
grouped.drop([''], inplace = True)
grouped.loc[grouped.shape[0]] = grouped.sum()
grouped.rename(index = {grouped.shape[0]-1: '合计总数'}, inplace = True)

# round up to precision with 2 digits
grouped = grouped.round(2)

output_filename = folder_name+'/统计入库_供应商_总金额.xlsx'
writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
grouped.to_excel(writer,'sheet1') #creates workbook 

# set data format
title = '入库统计总金额'+month
handle_worksheet_format(writer, grouped, title)
           
writer.save() #saves workbook to file in python file directory
writer.close()
grouped

In [ ]:
print('success')